<a href="https://colab.research.google.com/github/AbhiramNallamothu/Assignments-5731/blob/main/Abhiram_Nallamothu_Assignment_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Wednesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Function to fetch reviews from IMDB
def fetch_reviews(movie_id, max_reviews=1000):
    reviews = []
    current_page = 1
    while len(reviews) < max_reviews:
        url = f'https://www.imdb.com/title/{movie_id}/reviews?ref_=tt_ql_3&sort=submissionDate&dir=desc&ratingFilter=0&paginationKey={current_page}'
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extracting the reviews
        review_blocks = soup.find_all('div', class_='lister-item-content')
        for block in review_blocks:
            # Check if we've reached our review limit
            if len(reviews) >= max_reviews:
                break

            # Extract review title
            title = block.find('a', class_='title').text.strip()

            # Extract review text
            review_text = block.find('div', class_='text show-more__control').text.strip()

            # Append to our reviews list
            reviews.append({'title': title, 'review': review_text})

        # Check if there is next page
        next_page = soup.find('div', class_='load-more-data')
        if next_page:
            current_page = next_page['data-key']
        else:
            break  # No more pages

        # Sleep to avoid getting blocked by the server
        time.sleep(1)

    return reviews

movie_id = 'tt1375666'  # Inception
reviews = fetch_reviews(movie_id, 1000)

# Save the reviews to a CSV file
df = pd.DataFrame(reviews)
df.to_csv('imdb_reviews.csv', index=False)
print(f'Saved {len(reviews)} reviews to imdb_reviews.csv')



Saved 1000 reviews to imdb_reviews.csv


# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [2]:
# Write code for each of the sub parts with proper comments.
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Load the dataset
df = pd.read_csv('imdb_reviews.csv')

# Define a function to clean the text
def clean_text(text):
    # Remove special characters and punctuations
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Lowercase all texts
    text = text.lower()

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])

    # Stemming
    stemmer = PorterStemmer()
    text = ' '.join([stemmer.stem(word) for word in text.split()])

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

    return text

# Clean the 'review' column and create a new one for the cleaned data
df['clean_review'] = df['review'].apply(clean_text)

# Save the updated dataframe to a new CSV file
clean_csv_path = 'clean_imdb_reviews.csv'
df.to_csv(clean_csv_path, index=False)
print(f'Saved cleaned reviews to {clean_csv_path}')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Saved cleaned reviews to clean_imdb_reviews.csv


# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [3]:
!pip install spacy
!pip install pandas
!pip install nltk
!python -m spacy download en_core_web_sm
# Import necessary libraries
import pandas as pd
import spacy
from spacy import displacy
from collections import Counter
import nltk
from nltk.tree import Tree
from IPython.display import display

# Ensure you have the necessary NLTK datasets
nltk.download('punkt')

# Load Spacy's English language model
nlp = spacy.load("en_core_web_sm")

# Load the cleaned dataset
df = pd.read_csv('clean_imdb_reviews.csv')  # Make sure the file path is correct

# Initialize counters for POS tags
pos_counts = {'NOUN': 0, 'VERB': 0, 'ADJ': 0, 'ADV': 0}

# Named Entity Recognition counts
ner_counts = Counter()

# Function for POS tagging and NER
def analyze_text(text):
    doc = nlp(text)
    # POS Tagging
    for token in doc:
        if token.pos_ in pos_counts:
            pos_counts[token.pos_] += 1

    # Named Entity Recognition
    for ent in doc.ents:
        ner_counts[ent.label_] += 1

    # Dependency Parsing (displayed inline for each sentence)
    for sent in doc.sents:
        displacy.render(sent, style='dep', options={'distance': 90})

# Analyze each review (you might want to limit this to a subset for quicker results)
for review in df['clean_review'].head(10):  # Analyzing first 10 reviews for brevity
    analyze_text(review)

# Print POS counts
print("Parts of Speech Counts:", pos_counts)

# Print NER counts
print("Named Entity Counts:", ner_counts)

# Select an example sentence for parsing demonstration
example_sentence = df['clean_review'][0]  # Assuming there is at least one review
print("Example Sentence for Parsing:", example_sentence)

# Display Dependency Parsing for the example sentence
example_doc = nlp(example_sentence)
displacy.render(next(example_doc.sents), style='dep', jupyter=True, options={'distance': 90})

# Explanation of Constituency and Dependency Parsing for one sentence
print("""
Dependency Parsing Tree: Shows the relationships between words in a sentence. Each node is a word, and the edges represent the grammatical relationships between them. This helps in understanding how words in a sentence relate to each other.
""")

# Note: Constituency parsing typically requires additional setup and resources (like Stanford CoreNLP).
# For simplicity and resource constraints in a Jupyter/Colab environment, this example focuses on dependency parsing.



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 72.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Parts of Speech Counts: {'NOUN': 444, 'VERB': 131, 'ADJ': 173, 'ADV': 33}
Named Entity Counts: Counter({'PERSON': 40, 'ORG': 19, 'NORP': 8, 'CARDINAL': 3, 'DATE': 3, 'ORDINAL': 2, 'GPE': 2, 'LOC': 1, 'WORK_OF_ART': 1, 'PRODUCT': 1})
Example Sentence for Parsing: titl incept cool dream adventuredirector christoph nolan cast leonardo dicaprio joseph gordonlevitt ellen page tom hardi ken watanab marion cotillard cillian murphi michael cainer incept super fun movi peopl go dream main guy cobb realli good want go home famili sad memori wife need one last dream job make happencobb friend go big adventur differ dream need put special idea someon head easi face lot cool challeng along wayth actor amaz leonardo dicaprio play cobb smart dream guy joseph gordonlevitt arthur funni friend ellen page ariadn girl make cool dream world mani cool actor



Dependency Parsing Tree: Shows the relationships between words in a sentence. Each node is a word, and the edges represent the grammatical relationships between them. This helps in understanding how words in a sentence relate to each other.



# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [ ]:
# Write your response below
Extracting data has been a bit difficult task and all the other processes like lemmatization, stemming has inbuilt methods which made them work a bit easier
I had to spend some extra time to understand the working of individual modules.
Had to setup few librarires which consumed more time.
While web scrapping the data every website has their individual policies undersatanding them and writing code without probhiting their policies abnd getting data is bit diffuclt task in this assignment.